In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8534838361088915575, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15306329601180315333
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time

import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
from sklearn.metrics import accuracy_score
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
output_classes = 2
batch_size = 64
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
# epochs = 50

nb_train_samples = 787
nb_validation_samples = 262
nb_test_samples = 317

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split=0.25)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    subset="training",
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    subset="validation",
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 787 images belonging to 2 classes.
Found 262 images belonging to 2 classes.
Found 317 images belonging to 2 classes.


In [8]:
def vgg19_model():
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    prediction = Dense(output_classes, activation=tf.nn.softmax)(x)

    model = Model(inputs=base_model.input,outputs=prediction)
    return model

In [9]:
space = {
         'lr': hp.choice('lr',[1.0, 0.001, 0.0001, 0.00001,0.000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
         'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
         'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [10]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

#     random_seed = np.random.seed(1132)

#     train_datagen = ImageDataGenerator(
#         rescale=1. / 255,
#         featurewise_center=True,
#         featurewise_std_normalization=True)

#     test_datagen = ImageDataGenerator(rescale=1. / 255)

#     train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     validation_generator = train_datagen.flow_from_directory(
#         validation_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     test_generator = test_datagen.flow_from_directory(
#         test_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = False,
#         class_mode='categorical')

   
    adam_opt = Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = vgg19_model()
    
    model.compile(optimizer = adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      epochs = 15,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,
      verbose = 1)
    
    score = model.evaluate_generator(validation_generator, steps=50)

    print ('Validation Score: ', score[0])
    print ('Validation Accuracy: ',score[1])
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
    predict_class = np.argmax(predicts, axis=1)
    errors = np.where(predict_class != truth)[0]
    print("No of errors = {}/{}".format(len(errors),test_generator.samples))
    th= 0.3
    acc = accuracy_score(truth, predict_class > th)
    print("Accuracy: {:.4f}".format(acc*100))
    print("*_*" * 50)
    
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [11]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'lr': 1e-06, 'beta_1': 0.4, 'amsgrad': True, 'beta_2': 0.999}
Epoch 1/15
12/12 [==============================] - 18s 1s/step - loss: 0.8969 - acc: 0.6687 - val_loss: 0.4794 - val_acc: 0.9258
Epoch 2/15
12/12 [==============================] - 7s 613ms/step - loss: 0.7634 - acc: 0.7215 - val_loss: 0.4414 - val_acc: 0.9495
Epoch 3/15
12/12 [==============================] - 7s 602ms/step - loss: 0.6786 - acc: 0.7573 - val_loss: 0.3916 - val_acc: 0.9747
Epoch 4/15
12/12 [==============================] - 7s 607ms/step - loss: 0.6302 - acc: 0.7869 - val_loss: 0.4091 - val_acc: 0.9545
Epoch 5/15
12/12 [==============================] - 7s 618ms/step - loss: 0.6506 - acc: 0.7786 - val_loss: 0.3892 - val_acc: 0.9596
Epoch 6/15
12/12 [==============================] - 7s 575ms/step - loss: 0.5580 - acc: 0.8187 - val_loss: 0.3605 - val_acc: 0.9648
Epoch 7/15
12/12 [==============================] - 7s 615ms/step - loss: 0.5376 - acc: 0.8225 - val_loss: 0.3376 - val_acc: 0

12/12 [==============================] - 7s 623ms/step - loss: 9.2136 - acc: 0.6250 - val_loss: 9.9214 - val_acc: 0.6364
Epoch 9/15
12/12 [==============================] - 7s 562ms/step - loss: 9.4637 - acc: 0.6832 - val_loss: 7.8940 - val_acc: 0.6919
Epoch 10/15
12/12 [==============================] - 7s 612ms/step - loss: 9.0619 - acc: 0.6406 - val_loss: 8.1732 - val_acc: 0.6414
Epoch 11/15
12/12 [==============================] - 7s 590ms/step - loss: 9.7631 - acc: 0.6678 - val_loss: 9.1429 - val_acc: 0.6523
Epoch 12/15
12/12 [==============================] - 7s 606ms/step - loss: 11.2060 - acc: 0.6655 - val_loss: 8.2328 - val_acc: 0.6768
Epoch 13/15
12/12 [==============================] - 7s 600ms/step - loss: 9.0635 - acc: 0.6504 - val_loss: 9.5050 - val_acc: 0.6616
Epoch 14/15
12/12 [==============================] - 7s 599ms/step - loss: 9.0999 - acc: 0.6506 - val_loss: 9.5917 - val_acc: 0.6566
Epoch 15/15
12/12 [==============================] - 7s 579ms/step - loss: 11.778

12/12 [==============================] - 7s 600ms/step - loss: 0.1949 - acc: 0.9895 - val_loss: 1.8050 - val_acc: 0.5312
Epoch 7/15
12/12 [==============================] - 8s 667ms/step - loss: 0.2005 - acc: 0.9922 - val_loss: 0.2015 - val_acc: 0.9949
Epoch 8/15
12/12 [==============================] - 8s 631ms/step - loss: 0.1857 - acc: 0.9922 - val_loss: 0.1719 - val_acc: 1.0000
Epoch 9/15
12/12 [==============================] - 8s 631ms/step - loss: 0.2067 - acc: 0.9869 - val_loss: 1.7197 - val_acc: 0.6717
Epoch 10/15
12/12 [==============================] - 7s 614ms/step - loss: 0.2152 - acc: 0.9843 - val_loss: 0.2758 - val_acc: 0.9596
Epoch 11/15
12/12 [==============================] - 7s 621ms/step - loss: 0.1918 - acc: 0.9893 - val_loss: 0.1969 - val_acc: 0.9922
Epoch 12/15
12/12 [==============================] - 8s 642ms/step - loss: 0.1694 - acc: 1.0000 - val_loss: 0.1675 - val_acc: 1.0000
Epoch 13/15
12/12 [==============================] - 8s 633ms/step - loss: 0.1702 - 

ResourceExhaustedError: OOM when allocating tensor with shape[64,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_11/Adam/gradients/block1_conv2_11/convolution_grad/Conv2DBackpropInput = Conv2DBackpropInput[T=DT_FLOAT, _class=["loc:@train...propFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_11/Adam/gradients/block1_conv2_11/convolution_grad/ShapeN, block1_conv2_11/kernel/read, training_11/Adam/gradients/block1_conv2_11/Relu_grad/ReluGrad)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
